# "Duo Net"

In [2]:
import keras
from keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
import sys
sys.path.append('../')
from utils import DataFeed
from keras import regularizers

Using TensorFlow backend.


### Load Data

In [2]:
data_path='../preprocessing/preprocessed_data'

In [3]:
train_data, train_labels = DataFeed.Dataset.create(data_path, ['train/voxforge', 'train/youtube'], num=50000, use_premade=True)
val_data, val_labels = DataFeed.Dataset.create(data_path, ['val/youtube', 'val/voxforge'], num=-1, shuffle=True)

In [2]:
callback_stopearly = keras.callbacks.EarlyStopping(monitor='val_acc',
                                                   patience=5)

### Define Model

In [3]:
model = models.Sequential()
model.add(Melspectrogram(n_dft=512, input_shape=(1, 5 * 16000,),
                         padding='same', sr=16000, n_mels=28,
                         fmin=0.0, fmax=10000, power_melgram=1.0,
                         return_decibel_melgram=False, trainable_fb=False,
                         trainable_kernel=False))
model.add(layers.SeparableConv2D(128, (5, 1), activation='relu'))
model.add(layers.MaxPooling2D((3, 1)))
model.add(layers.SeparableConv2D(128, (5, 1), activation='relu'))
model.add(layers.MaxPooling2D((4, 1)))
model.add(layers.Reshape((-1, 64)))
model.add(layers.SeparableConv1D(128, 14, activation='relu'))
model.add(layers.MaxPooling1D(4))
model.add(layers.SeparableConv1D(128, 14, activation='relu'))
model.add(layers.MaxPooling1D(4))
model.add(layers.SeparableConv1D(128, 14, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Flatten())
model.add(layers.Dense(3, activation='softmax'))

In [72]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melspectrogram_38 (Melspectr (None, 28, 313, 1)        270364    
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 24, 313, 64)       384       
_________________________________________________________________
max_pooling2d_75 (MaxPooling (None, 8, 313, 64)        0         
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 4, 313, 64)        20544     
_________________________________________________________________
max_pooling2d_76 (MaxPooling (None, 1, 313, 64)        0         
_________________________________________________________________
reshape_37 (Reshape)         (None, 313, 64)           0         
_________________________________________________________________
conv1d_92 (Conv1D)           (None, 300, 128)          114816    
__________

In [81]:
model.compile(optimizer='Rmsprop',
              metrics=['accuracy'],
              loss='categorical_crossentropy')

In [13]:
a = np.array([[1, 2], [3, 4]])

In [15]:
a

array([[1, 2],
       [3, 4]])

In [22]:
a[:, :, None].shape

(2, 2, 1)